In [1]:
import os
import json

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np

import s3fs
from datetime import datetime
from datetime import timedelta
import calendar

In [2]:
raw_test = pd.read_csv('s3://msds630-kaggle-competition/sessions_events_modified_v2.csv/part-00000', header=None)

In [3]:
columns = ["session_id", "start_timestamp","time_zone","timezone_offset","previous_sessions","user_created_stamp","is_first_session","country", "region","city","latitude","longitude","local","os_name","session_index","device_id","user_id_hash","num_events","num_purchases","weekend_purchase","is_purchase",'purchase_times','sum_purchase','1week','2week']
raw_test.columns = columns

In [4]:
import math

def convertTime(ts_str):
    if math.isnan(float(ts_str)):
        return None
    else:
        return datetime.utcfromtimestamp(int(ts_str)/1000.0).strftime('%Y-%m-%d %H:%M:%S')
convertTime('1541638424150')

'2018-11-08 00:53:44'

In [5]:
raw_test['start_timestamp'] = raw_test['start_timestamp'].map(lambda x: convertTime(x))
columns = ["user_id_hash","session_id", "start_timestamp","time_zone","timezone_offset","previous_sessions","user_created_stamp","is_first_session","country", "region","city","latitude","longitude","local","os_name","session_index","device_id","num_events","num_purchases","weekend_purchase","is_purchase",'purchase_times','sum_purchase','1week','2week']
raw_test = raw_test.reindex(columns,axis=1)

In [6]:
raw_test.drop(["sum_purchase"], axis=1, inplace=True)

In [7]:
raw_train = raw_test[raw_test["start_timestamp"] < "2018-12-02"] 

In [8]:
raw_test.head()

,user_id_hash,session_id,start_timestamp,time_zone,timezone_offset,previous_sessions,user_created_stamp,is_first_session,country,region,...,os_name,session_index,device_id,num_events,num_purchases,weekend_purchase,is_purchase,purchase_times,1week,2week
0,9e095d1e1a4049b2c830d3884ca526c3cd00faadef7768...,6282115060269174490,2018-10-28 06:52:14,America/Los_Angeles,-25200000.0,127776870.0,1.539629e+12,False,US,wa,...,Android OS,52.0,eaf0f10b-5e67-412a-9a30-7a687d902636,11.0,0.0,NaN,-1.0,NaN,-1.0,-1.0
1,965bbf8946615acfd53b9d4febc4bfb8f76d2ca8dcc47e...,3311962070546841600,2018-10-18 00:42:48,America/New_York,-14400000.0,0.0,1.539823e+12,True,US,nc,...,Android OS,1.0,f90df33e-5bd3-41b0-af5e-3613bfaff7df,57.0,0.0,NaN,-1.0,NaN,-1.0,-1.0
2,8ef37a8bf91fcf32fd2dff7361140d062d25e89403498c...,2227925547996687280,2018-11-17 16:41:00,America/New_York,-18000000.0,148898.0,1.542405e+12,False,US,ny,...,iOS,3.0,779DA08D-B10E-45EC-89FB-C28D15E02C88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,8939252064915685380,2018-11-28 10:36:47,Australia/Sydney,39600000.0,61598736.0,1.540796e+12,False,AU,vic,...,Android OS,58.0,f8ae3393-8a50-426c-8e61-b855478f4de0,4.0,0.0,NaN,-1.0,NaN,-1.0,-1.0
4,b8f647bd603c347e013739f5c3bb9568f21d6736c82fb9...,5558769389058741590,2018-11-13 20:07:28,Asia/Kolkata,19800000.0,17663056.0,1.541154e+12,False,IN,up,...,iOS,30.0,400E3C4E-55B6-4BAC-AEB9-12C0F9EB2FC6,42.0,0.0,NaN,-1.0,NaN,-1.0,-1.0


In [9]:
raw_train.head()

,user_id_hash,session_id,start_timestamp,time_zone,timezone_offset,previous_sessions,user_created_stamp,is_first_session,country,region,...,os_name,session_index,device_id,num_events,num_purchases,weekend_purchase,is_purchase,purchase_times,1week,2week
0,9e095d1e1a4049b2c830d3884ca526c3cd00faadef7768...,6282115060269174490,2018-10-28 06:52:14,America/Los_Angeles,-25200000.0,127776870.0,1.539629e+12,False,US,wa,...,Android OS,52.0,eaf0f10b-5e67-412a-9a30-7a687d902636,11.0,0.0,NaN,-1.0,NaN,-1.0,-1.0
1,965bbf8946615acfd53b9d4febc4bfb8f76d2ca8dcc47e...,3311962070546841600,2018-10-18 00:42:48,America/New_York,-14400000.0,0.0,1.539823e+12,True,US,nc,...,Android OS,1.0,f90df33e-5bd3-41b0-af5e-3613bfaff7df,57.0,0.0,NaN,-1.0,NaN,-1.0,-1.0
2,8ef37a8bf91fcf32fd2dff7361140d062d25e89403498c...,2227925547996687280,2018-11-17 16:41:00,America/New_York,-18000000.0,148898.0,1.542405e+12,False,US,ny,...,iOS,3.0,779DA08D-B10E-45EC-89FB-C28D15E02C88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1b24599435b469b251f4f6c907a46d54b334e956e815a0...,8939252064915685380,2018-11-28 10:36:47,Australia/Sydney,39600000.0,61598736.0,1.540796e+12,False,AU,vic,...,Android OS,58.0,f8ae3393-8a50-426c-8e61-b855478f4de0,4.0,0.0,NaN,-1.0,NaN,-1.0,-1.0
4,b8f647bd603c347e013739f5c3bb9568f21d6736c82fb9...,5558769389058741590,2018-11-13 20:07:28,Asia/Kolkata,19800000.0,17663056.0,1.541154e+12,False,IN,up,...,iOS,30.0,400E3C4E-55B6-4BAC-AEB9-12C0F9EB2FC6,42.0,0.0,NaN,-1.0,NaN,-1.0,-1.0


In [10]:
def enconder(x):
    if x==1: return x
    else: return 0

In [11]:
raw_test['1week'] = raw_test['1week'].map(lambda x: enconder(x))
raw_test['2week'] = raw_test['2week'].map(lambda x: enconder(x))

In [12]:
raw_test['session_id'] = raw_test['session_id'].map(lambda x: 1)

In [13]:
new = raw_test.groupby('user_id_hash').sum()

In [14]:
def encoder2(x):
    if x>0: return 1
    else: return 0

In [15]:
new['1week'] = new['1week'].map(lambda x: encoder2(x))
new['2week'] = new['2week'].map(lambda x: encoder2(x))

In [16]:
new['1week'].value_counts()

0    615121
1      4398
Name: 1week, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new[new.columns[:-1]], 
                                                    new['2week'], test_size=0.10)

In [18]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

/anaconda3/envs/msds694/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     61414
           1       0.36      0.02      0.04       538

   micro avg       0.99      0.99      0.99     61952
   macro avg       0.68      0.51      0.52     61952
weighted avg       0.99      0.99      0.99     61952



In [20]:
logmodel.fit(new[new.columns[:-1]],new['2week'])
predictions = logmodel.predict(new[new.columns[:-1]])

/anaconda3/envs/msds694/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
#### Predictions

In [22]:
pred = pd.DataFrame(list(map(list, zip(*[new.index.tolist(),list(predictions)]))))

In [23]:
df = pd.read_csv("Data/sample_submission_2.csv")

In [24]:
def enco_pred(x):
    if x in pred.values:
        pred[pred["user_id_hash"]==x]['1week'].values[0]
    else:
        return 0

In [34]:
pred.columns = ["user_id_hash","prediction"]

In [35]:
pred.head()

,user_id_hash,prediction
0,00000db35682058bb7916cb90f85709f54c1a0f7a3b6de...,0
1,000059859ec188af6035870faf885c3038cedda05b3a54...,0
2,000062e9be78f3da274fec338e78f89d12000e781967f2...,0
3,00006ad44c0d33a2a526caf75514643639b7b7070dfffd...,0
4,000081a7d841b0f953bdba6c185fd5cc335fd6015a30c3...,0


In [36]:
df.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.01,0.02
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.01,0.02


In [37]:
new2 = df.merge(pred, on='user_id_hash', how="left")

In [38]:
new2.drop('user_purchase_binary_7_days', inplace=True, axis=1)

In [42]:
new2['user_purchase_binary_14_days'] = new2['prediction']

In [43]:
def rand(x):
    if x == 1:
        return x
    else:
        return np.random.choice((1,0), p=[0.23, 0.77])

In [44]:
new2['user_purchase_binary_14_days'] = new2['user_purchase_binary_14_days'].map(lambda x: rand(x))

In [45]:
new2.columns = ['user_id_hash', 'user_purchase_binary_14_days', 'user_purchase_binary_7_days']

In [46]:
columns = ['user_id_hash', 'user_purchase_binary_7_days', 'user_purchase_binary_14_days']

In [47]:
new2 = new2.reindex(columns,axis=1)

In [48]:
new2['user_purchase_binary_7_days'] = new2['user_purchase_binary_7_days'].map(lambda x: enconder(x))

In [49]:
new2.count()

user_id_hash                    312568
user_purchase_binary_7_days     312568
user_purchase_binary_14_days    312568
dtype: int64

In [50]:
new2.to_csv('submission1.csv', index=False)